In [1]:
import sys
sys.path.append("/Users/msachde1/Downloads/Research/Development/mgwr/")

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from mgwr.gwr import GWR
import pandas as pd
import numpy as np
from spglm.family import Gaussian, Binomial, Poisson
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
import multiprocessing as mp
pool = mp.Pool()
import numpy as np
from scipy import linalg
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse import linalg as spla
from spreg.utils import spdot, spmultiply
from scipy import special
import libpysal as ps
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import copy
from collections import namedtuple

\begin{align}
y = Poisson[E_i exp ({\sum} {\beta} & _k x _{k,i})]\\
E_i = Offset\\
\hat{y} = predicted(y)\\
z = ({\sum} {\beta} & _k x _{k,i}) + ((y - \hat{y})/\hat{y})\\
\end{align}

### IWLS convergence loop

In [103]:
data_p = ps.io.open(ps.examples.get_path('Tokyomortality.csv'))
coords = list(zip(data_p.by_col('X_CENTROID'),data_p.by_col('Y_CENTROID')))
off = np.array(data_p.by_col('eb2564')).reshape((-1,1))
y = np.array(data_p.by_col('db2564')).reshape((-1,1)) 
occ = np.array(data_p.by_col('OCC_TEC')).reshape((-1,1))
own = np.array(data_p.by_col('OWNH')).reshape((-1,1))
pop = np.array(data_p.by_col('POP65')).reshape((-1,1))
unemp = np.array(data_p.by_col('UNEMP')).reshape((-1,1))
X = np.hstack([occ,own,pop,unemp])
x = pop

X_std = (X-X.mean(axis=0))/X.std(axis=0)
x_std = (x-x.mean(axis=0))/x.std(axis=0)
y_std = (y-y.mean(axis=0))/y.std(axis=0)

In [104]:
n_iter=0
k=x.shape[0]

### Initialization with GWPR

In [105]:
sel=Sel_BW(coords,y_std,x_std,family=Poisson(),offset=off)
bw_in=sel.search()
def gwr_func(y,X,bw):
    return GWR(coords,y,X,bw,family=Poisson(),offset=off,fixed=False,kernel='bisquare').fit()
optim_model = gwr_func(y=y,X=x,bw=bw_in)
om_p=optim_model.params

### Starting values

In [106]:
def weights(mu):
    deriv_p = 1.0/mu
    variance = np.ones(mu.shape,np.float64)
    return 1.0/(deriv_p**2 * variance)

In [107]:
def beta_sum(x,param,k):
    cons=spdot(np.ones(k),param[:,0])
    b=np.delete(param,[0],axis=1)
    return cons+(spdot(np.array(x).reshape(-1),b))

In [108]:
v=beta_sum(x,om_p,k)
yhat=(off*np.exp(v))

In [109]:
diff = 1.0e+06
tol = 1.0e-06
max_iter=200
betas=om_p

In [110]:
while diff> tol and n_iter < max_iter:
    n_iter +=1
    w = weights(yhat)
    z=v+((y-yhat)/yhat)
    w = np.sqrt(w)

    wx = spmultiply(x,w,array_out=False)
    wz = spmultiply(z,w,array_out=False)
    
    wz_std=(wz-wz.mean(axis=0))/wz.std(axis=0)
    wx_std=(wx-wx.mean(axis=0))/wx.std(axis=0)
    
    selector=Sel_BW(coords,wz_std,wx_std,multi=True)
    selector.search(pool=pool)
    print(selector.bw[0])
    mgwr_model=MGWR(coords,wz_std,wx_std,selector).fit()
    
    n_betas=mgwr_model.params 

    v=beta_sum(x_std,n_betas,k)
    yhat=(off*np.exp(v))
    
    diff=min(min(abs(betas-n_betas).reshape(1,-1).tolist()))
    print("diff = "+str(diff))
    betas = n_betas

#print (betas, yhat, v, w, z, n_iter)

[43. 43.]



diff = 0.147808394767278
[43. 44.]



diff = 3.654703760913591e-05
[43. 44.]



diff = 3.2586819020963276e-08
